# Geschwindigkeit Tests
Die Klasse `Cache` aus dem Notebook `nmm-cache.ipynb`implementiert eine Transpositionstabelle die persistiert werden kann. Um zu ermitteln wie der Cache implementiert werden soll, wurde zwischen zwei Möglichkeiten abgewogen. Die eine Möglichkeit ist die Verwendung einer externen Key-Value-Datenbank, die als extra Container auf dem Computer ausgeführt wird. Die zweite Möglichkeit ist das direkte Zwischenspeichern im Arbeitsspeicher und das anschließende exportieren auf einen persistenten Datenträger.

Um zu ermitteln welche Methode verwendet werden sollte, wird im Nachfolgenden die Performance der jeweiligen Methode getestet und bewertet.

## Vorbereitungen
Zum einen müssen folgende Packete importiert werden:
- `redis` - Dieses Packet implementiert die Kommunikation mit der Redis-Datenbank
- `struct` - Wird verwendet um Float Werte in Bytes zu übersetzen
- `tqdm` - Wird verwendet um den Fortschritt anzuzeigen

Des weiteren wird eine Klasse `Cache` angelegt, die als Interface für die jeweilige Implementierung dient. Diese besitzt die Methoden `write` und `read` welche als Vorlage schreiben und auslesen des Caches dienen.

In [ ]:
import redis
import struct
import random
import resource
from tqdm import tqdm

In [ ]:
class Cache():
    def write(self, state: int, player: bool, limit: int, value: float, alpha: float, beta: float) -> None:
        pass
    
    def read(self, state: int, player: bool, limit: int) -> (float,float,float):
        """Value, Alpha, Beta"""
        pass


## Python Cache
Der Python Cache besteht aus einem Python-Dictionary in welches die Key-Value-Paare gespeichert werden.

In [ ]:
class Python_Cache(Cache):
    def __init__(self):
        self.cache = {}
        pass
    
    def write(self, state: int, player: bool, limit: int, value: float, alpha: float, beta: float) -> None:
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = struct.pack("d", value) + struct.pack("d",alpha) + struct.pack("d",beta)
        self.cache[key] = value
    
    def read(self, state: int, player: bool, limit: int) -> (float,float,float):
        """Value, Alpha, Beta"""
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = self.cache.get(key)
        return (
            struct.unpack("d", value[:8])[0],
            struct.unpack("d", value[8:16])[0],
            struct.unpack("d", value[16:24])[0],
        ) if value else None

## Redis Cache
Der Redis-Cache implementiert einen Transpositionstabelle auf basierend auf der Key-Value-Datenbank Redis. Die Übersetzung der Werte zu Byte-Arrays erfolgt equivalent zu dem Python-Cache. Nur die Werte werden letzendlich mithilfe des `redis` Packets in der lokalen Redis-Datenbank gespeichert.

In [ ]:
class Redis_Cache(Cache):
    r = redis.Redis(host='redis')
    def write(self, state: int, player: bool, limit: int, value: float, alpha: float, beta: float) -> None:
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = struct.pack("d",value) + struct.pack("d",alpha) + struct.pack("d",beta)
        self.r.set(key,value)

    def read(self, state: int, player: bool, limit: int):
        key = state.to_bytes(8,'big') + player.to_bytes(1,'big') + limit.to_bytes(1,'big')
        value = self.r.get(key)
        return (
            struct.unpack("d", value[:8])[0],
            struct.unpack("d", value[8:16])[0],
            struct.unpack("d", value[16:24])[0],
        ) if value else None

## Test
Um den Test durchführen zu können, werden zuerst die einzelnen Parameter definiert und die jeweiligen Caches instanziiert. Insgesamt werden 1 000 000 Werte zum Testen verwendet.

In [ ]:
pc = Python_Cache()
rc = Redis_Cache()

start = 1000000000
count = 1000000
end   = start+count

In [ ]:
resource.getrusage(resource.RUSAGE_SELF)

### Test der Schreibgeschwindigkeit
Ingesamt werden dem Parameter `count` entsprechend viele Einträge in den Cache geschrieben. Die Werte werden dabei zufällig generiert.

In [ ]:
%%time
print('Python Cache')
for state in tqdm(range(start, end)):
    for player in [True, False]:
        for limit in range(4):
            pc.write(state, player, limit, random.random(), random.random(), -random.random())



In [ ]:
%%time
print('Redis Cache')
for state in tqdm(range(start, end)):
    for player in [True, False]:
        for limit in range(4):
            rc.write(state, player, limit, random.random(), random.random(), -random.random())

Im Vergleich zu dem `Python_Cache` ist der `Redis_Cache` beim Schreiben der Einträge etwa um den Faktor '90' langsamer.


In [ ]:
resource.getrusage(resource.RUSAGE_SELF)

### Testen der Lesegeschwindigkeit
Bei dem Test der Lesegeschwindigkeit wird gemessen, wie schnell die Werte für zufällig ermittelte Schlüssel zurück gegeben werden. Dabei wird auf die weiter oben generierten Einträge zurück gegriffen.

In [ ]:
%%time
valid = 0
for count in tqdm(range(count)):
    state = random.randint(start, end-1)
    limit = random.randint(0, 3)
    player = random.choice([True, False])
    result = pc.read(state, player, limit)
    if result is None:
        print(state, player, limit)
    else:
        valid += 1
print(valid)

In [ ]:
%%time
valid = 0
for count in tqdm(range(count)):
    state = random.randint(start, end-1)
    limit = random.randint(0, 3)
    player = random.choice([True, False])
    result = rc.read(state, player, limit)
    if result is None:
        print(state, player, limit)
    else:
        valid += 1
print(valid)

Währnd die Lesegeschwindigkeit beim `Redis_Cache` deutlich höher als die Schreibgeschwindigkeit ist, ist diese dennoch um etwa den Faktor vier langsamer als die des `Python_Caches`.

In [ ]:
resource.getrusage(resource.RUSAGE_SELF)

## Resumee
Sowohl der bei der Lese- als auch bei der Schreibgeschwindigkeit ist der `Python_Cache` deutlich schneller als der `Redis_Cache`. Wobei die Geschwindigkeits-Differenz beim Schreiben deutlich höher ist als beim Lesen. Dies ist höchst wahrscheinlich auf den Verbindungs-Overhead zwischen Datenbank und dem Python-Programm zurück zu führen. Somit wurde sich für dafür entschieden, den Cache als Python-Dictionary zu implementieren und anschließend als Datei auf dem Datenträger abzulegen. Hierbei wird in kauf genommen, dass es mit wachsender Größe des Caches zu einer starkenAuslastung des Arbeitsspeicher kommen wird.